## SFT

## Simple SFT

```bash
# unsloth 有个bug，会不认 Huggingface 的 model cache, 所以需要手动下载
huggingface-cli download --resume-download Qwen/Qwen2.5-1.5B-Instruct --local-dir ~/models/Qwen2.5-1.5B-Instruct
```

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/cynic/models/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    local_files_only=True,
    # fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    # gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank * 2,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-20 02:25:23 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

In [9]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="data/train_sft_simple.jsonl")["train"]

In [16]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["messages"])
    texts = [tokenizer.decode(text) for text in texts]
    return { "text" : texts }

dataset = dataset.map(apply_chat_template, batched = True, load_from_cache_file=False)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [18]:
print(dataset[100]["text"])

<|im_start|>system
You are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.<|im_end|>
<|im_start|>user
Using the numbers 40, 7, 356, 18, create an equation that equals 385. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once, and you must use all the numbers. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>(1 + 2) / 3</answer>. Think step by step inside <think> tags.<|im_end|>
<|im_start|>assistant
<think>Step 1: 40 + 7 = 47
Step 2: 47 + 356 = 403
Step 3: 403 - 18 = 385
Final answer: ((40 + 7) + 356) - 18</think>

<answer>((40 + 7) + 356) - 18</answer><|im_end|>



In [19]:
import wandb
wandb.init(project="countdown-sft-simple")


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [20]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=32):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
print(tokenizer.decode(trainer.train_dataset[100]["input_ids"]))

<|im_start|>system
You are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.<|im_end|>
<|im_start|>user
Using the numbers 40, 7, 356, 18, create an equation that equals 385. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once, and you must use all the numbers. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>(1 + 2) / 3</answer>. Think step by step inside <think> tags.<|im_end|>
<|im_start|>assistant
<think>Step 1: 40 + 7 = 47
Step 2: 47 + 356 = 403
Step 3: 403 - 18 = 385
Final answer: ((40 + 7) + 356) - 18</think>

<answer>((40 + 7) + 356) - 18</answer><|im_end|>



In [ ]:
# todo: train completions only

In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 375
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072/1,617,573,376 (4.57% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,2.053700
2,2.055800
3,1.689400
4,1.273900
5,0.938200
6,0.609800
7,0.313000
8,0.187700
9,0.129200
10,0.133200


In [24]:
model.save_pretrained("output/qwen2.5-1.5b-sft-simple-lora")  # Local saving lora weights
tokenizer.save_pretrained("output/qwen2.5-1.5b-sft-simple-lora")

('output/qwen2.5-1.5b-sft-simple-lora/tokenizer_config.json',
 'output/qwen2.5-1.5b-sft-simple-lora/special_tokens_map.json',
 'output/qwen2.5-1.5b-sft-simple-lora/vocab.json',
 'output/qwen2.5-1.5b-sft-simple-lora/merges.txt',
 'output/qwen2.5-1.5b-sft-simple-lora/added_tokens.json',
 'output/qwen2.5-1.5b-sft-simple-lora/tokenizer.json')

vllm inference
```bash
vllm serve Qwen/Qwen2.5-1.5B-Instruct --port 8100 --api-key NLUKKXIJDZ91rpg1z --enforce-eager  --max-model-len 4096 --enable-lora --max-lora-rank 64 --lora-modules qwen2.5-1.5b-sft-simple-lora=output/qwen2.5-1.5b-sft-simple-lora

CURATOR_VIEWER=1 python eval.py --provider vllm --data_path data/test.jsonl --model_name qwen2.5-1.5b-sft-simple-lora --temperature 0.01 --max_tokens 1024

https://curator.bespokelabs.ai/datasets/ebbe5e92cee7442aaaed1726164d2204  

Accuracy: 19/100 (19.00%)
```

分析：

1. 格式上 SFT 后完全遵守，没有问题
2. 不再出现数字使用错误（包括没有使用或者使用超过一次）
3. 所有的错误都是计算错误，最终不相等


本质还是合成的简单推理过程过于简洁，模型没有学会推理。